In [133]:
import pandas as pd
import json
import os
import time

Read data files

In [134]:
pose_type = 'walking'
test_folder = f'test_data_30_05/{pose_type}'

# Get all files after the start file (including it)
files = os.listdir(test_folder)
files = sorted(files)

# start_file = '2024-05-22 21_44_15.135255.json'
# files = sorted(files)[files.index(start_file):]

In [135]:
# Collect all data points into one list
json_data = []
for file in files:
    with open(os.path.join(test_folder, file), 'r') as f:
        dic = json.load(f)
        json_data.extend(dic['data'])


Format time

In [136]:
start_time = json_data[0]['time']
end_time = json_data[-1]['time']

print(start_time)

for i in range(len(json_data)):
    json_data[i]['time'] = json_data[i]['time'] - start_time

# Total time of recorded data
print(time.strftime('%H:%M:%S', time.gmtime(json_data[-1]['time'])))

1717075196.3581448
00:06:16


Remove duplicate points (when data was updated too slowly)

In [137]:
# Remove points with duplicate time
json_data = [json_data[i] for i in range(len(json_data)) if i == 0 or json_data[i]['time'] != json_data[i-1]['time']]

Map dictionary items to bus numbers

In [138]:
devices = ["LEFT_ARM", "RIGHT_ARM", "LEFT_LEG", "RIGHT_LEG"]

device_data = dict.fromkeys(devices)

for d in devices:
    # Two buses
    # Two MPUs and one QMC, then one MPU and one QMC
    device_data[d] = [[[], [], []], [[], []]]

for point in json_data:
    for device in devices:
        # Add empty data points for the missing data
        for i in range(2):
            for j in range(3):
                if i == 1 and j == 2:
                    continue
                device_data[device][i][j].append(None)

        if device not in point:
            continue

        device_point = point[device]['data']
        
        if 'mpu' not in device_point:
            continue
        
        # Remove missing MPU data
        for i in range(0, len(device_point['mpu']), -1):
            if device_point['mpu'][i] is None:
                device_point['mpu'].pop(i)
        
        # Legs don't have qmc data
        if device_point['qmc'] is None:
            device_point['qmc'] = []
        
        mpu_len = len(device_point['mpu'])
        qmc_len = len(device_point['qmc'])

        # Assign the data points that exist
        if mpu_len > 0:
            device_data[device][0][0][-1] = device_point['mpu'][0]
        if mpu_len > 1:
            device_data[device][0][1][-1] = device_point['mpu'][1]
        if qmc_len > 0:
            device_data[device][0][2][-1] = device_point['qmc'][0]

        if mpu_len > 2:
            device_data[device][1][0][-1] = device_point['mpu'][2]
        if qmc_len > 1:
            device_data[device][1][1][-1] = device_point['qmc'][1]

In [139]:
print(f'# of missing points: {sum(1 for x in device_data[devices[0]][0][0] if x is None)}')
device_data[devices[0]][0][0]

# of missing points: 0


[{'q': [-316, 498, 58, -804],
  'e': [-123, 57, 24],
  'g': [-5000, -11000, 6000],
  'a': [82, -25, 349]},
 {'q': [897, -152, 413, -25],
  'e': [-7, -48, 22],
  'g': [2000, 7000, 5000],
  'a': [455, -686, 920]},
 {'q': [-226, 421, 127, -868],
  'e': [-152, 52, -2],
  'g': [4000, 1000, 24000],
  'a': [-628, -730, -91]},
 {'q': [664, 67, 443, -597],
  'e': [97, -30, -46],
  'g': [5000, 2000, -15000],
  'a': [176, 566, 810]},
 {'q': [-36, 366, 218, -903],
  'e': [172, 42, -30],
  'g': [-14000, -33000, -27000],
  'a': [174, 411, -265]},
 {'q': [-888, 265, -372, -39],
  'e': [-20, -39, 40],
  'g': [12000, 18000, 9000],
  'a': [990, 550, 533]},
 {'q': [-358, -285, -323, 827],
  'e': [126, 13, -49],
  'g': [-23000, -19000, 57000],
  'a': [-392, 365, -105]}]

Map bus numbers to location names

In [140]:
device_locations = {
    'LEFT_ARM': [['8 Acc LUA^ accX', '17 Acc BACK accX', '83 IMU LUA magneticX'], ['14 Acc LH accX', '96 IMU LLA magneticX']],
    'RIGHT_ARM': [['11 Acc RUA_ accX', '23 Acc RWR accX', '57 IMU RUA magneticX'], ['5 Acc HIP accX', '44 IMU BACK magneticX']],
    'LEFT_LEG': [['103 IMU L-SHOE EuX'], []],
    'RIGHT_LEG': [['20 Acc RKN_ accX', '119 IMU R-SHOE EuX'], []],
    # 'RIGHT_LEG': [['20 Acc RKN_ accX', '128 IMU R-SHOE AngVelBodyFrameX'], []],

    # FIXME: This is only for that run on 22/05 when the 2nd mpu was not connected
    # Either RKN or R-SHOE was lost. Need to check data and compare with L-SHOE, also check that stepping movement we did with each foot
    # 'R_LEG_NRF52': [['119 IMU R-SHOE EuX'], []],
}

acc_to_imu = {
    '8 Acc LUA^ accX': '77 IMU LUA accX',
    '17 Acc BACK accX': '38 IMU BACK accX',
    '11 Acc RUA_ accX': '51 IMU RUA accX',
}

is_euler = ['103 IMU L-SHOE EuX']

device_columns = {}

for device in device_locations:
    for i in range(len(device_locations[device])):
        for j in range(len(device_locations[device][i])):
            if device_locations[device][i][j] == '':
                continue
            
            # Normalize device data
            for k in range(len(device_data[device][i][j])):
                if device_data[device][i][j][k] is None:
                    device_data[device][i][j][k] = {}

                if 'q' not in device_data[device][i][j][k]:
                    device_data[device][i][j][k]['q'] = [None] * 4

                for key in ['e', 'a', 'g', 'm']:
                    if key not in device_data[device][i][j][k]:
                        device_data[device][i][j][k][key] = [None] * 3

            sample_point = device_data[device][i][j][0]
            initial_column_name = device_locations[device][i][j]
            
            initial_column_index = int(initial_column_name.split(' ')[0])
            initial_column_name = ' '.join(initial_column_name.split(' ')[1:])

            axes = ['X', 'Y', 'Z']
            
            # print(sample_point)
            # print([x['g'][0] if x['g'][0] is not None else None for x in device_data[device][i][j]])
            
            # for x in device_data[device][i][j]:
            #     if isinstance(x['g'], str):
            #         print(x)
            
            def add_columns(key):
                # try:
                for k in range(3):
                    device_columns[f'{initial_column_index + k} {initial_column_name[:-1] + axes[k]}'] = [round(x[key][k]) if x[key][k] is not None else None for x in device_data[device][i][j]]
                # except Exception as e:
                #     print(e)
                #     print(device_data[device][i][j])

            if 'a' in sample_point:
                # MPU

                if 'SHOE EuX' in initial_column_name:
                    # Euler angles
                    if 'e' in sample_point:
                        add_columns('e')
                    
                    # Skip Nav_A
                    initial_column_index += 3
                    
                    initial_column_name = initial_column_name.replace('Eu', 'Body_A')
                    initial_column_index += 3

                    # Body acceleration
                    axes = ['x', 'y', 'z']
                    add_columns('a')
                    axes = ['X', 'Y', 'Z']

                    initial_column_name = initial_column_name.replace('Body_A', 'AngVelBodyFrame')
                    initial_column_index += 3

                    # Body angular velocity
                    if 'g' in sample_point:
                        add_columns('g')

                    # Ignore Nav angular velocity and compass

                    continue

                # Acceleration
                add_columns('a')

                # IMU
                if device_locations[device][i][j] in acc_to_imu:
                    initial_imu_name = acc_to_imu[device_locations[device][i][j]]
                    
                    initial_imu_index = int(initial_imu_name.split(' ')[0])
                    initial_imu_name = ' '.join(initial_imu_name.split(' ')[1:])
                    
                    def add_imu_columns(key):
                        for k in range(3):
                            device_columns[f'{initial_imu_index + k} {initial_imu_name[:-1] + axes[k]}'] = [round(x[key][k]) if x[key][k] is not None else None for x in device_data[device][i][j]]

                    # Acceleration
                    add_imu_columns('a')
                    
                    initial_imu_name = initial_imu_name.replace('acc', 'gyro')
                    initial_imu_index += 3
                    
                    # Gyro
                    if 'g' in sample_point:
                        add_imu_columns('g')
                    
                    # Skip Magnetic
                    initial_imu_index += 3
                    
                    initial_imu_name = initial_imu_name.replace('gyro' + axes[0], 'Quaternion1')
                    initial_imu_index += 3
                    
                    # Quaternion
                    for k in range(4):
                        device_columns[f'{initial_imu_index + k} {initial_imu_name[:-1] + str(k+1)}'] = [round(x['q'][k]) if x['q'][k] is not None else None for x in device_data[device][i][j]]

            elif 'm' in sample_point:
                # QMC
                
                # Magnetic
                for k in range(3):
                    device_columns[f'{initial_column_index + k} {initial_column_name[:-1] + axes[k]}'] = [round(x['m'][k]) if x['m'][k] is not None else None for x in device_data[device][i][j]]



In [141]:
print(len(device_columns))
print(list(device_columns.keys()))
print(len(device_columns['83 IMU LUA magneticX']))

81
['8 Acc LUA^ accX', '9 Acc LUA^ accY', '10 Acc LUA^ accZ', '77 IMU LUA accX', '78 IMU LUA accY', '79 IMU LUA accZ', '80 IMU LUA gyroX', '81 IMU LUA gyroY', '82 IMU LUA gyroZ', '86 IMU LUA Quaternion1', '87 IMU LUA Quaternion2', '88 IMU LUA Quaternion3', '89 IMU LUA Quaternion4', '17 Acc BACK accX', '18 Acc BACK accY', '19 Acc BACK accZ', '38 IMU BACK accX', '39 IMU BACK accY', '40 IMU BACK accZ', '41 IMU BACK gyroX', '42 IMU BACK gyroY', '43 IMU BACK gyroZ', '47 IMU BACK Quaternion1', '48 IMU BACK Quaternion2', '49 IMU BACK Quaternion3', '50 IMU BACK Quaternion4', '83 IMU LUA magneticX', '84 IMU LUA magneticY', '85 IMU LUA magneticZ', '14 Acc LH accX', '15 Acc LH accY', '16 Acc LH accZ', '96 IMU LLA magneticX', '97 IMU LLA magneticY', '98 IMU LLA magneticZ', '11 Acc RUA_ accX', '12 Acc RUA_ accY', '13 Acc RUA_ accZ', '51 IMU RUA accX', '52 IMU RUA accY', '53 IMU RUA accZ', '54 IMU RUA gyroX', '55 IMU RUA gyroY', '56 IMU RUA gyroZ', '60 IMU RUA Quaternion1', '61 IMU RUA Quaternion2',

Add time column in ms

In [142]:
time_column = [round(x['time'] * 1000) for x in json_data]
print(time_column[:10])

[0, 65131, 125878, 193686, 257593, 314104, 376554]


Fill in all the missing columns with empty values

In [143]:
column_numbers = [int(x.split(' ')[0]) for x in device_columns.keys()]

for i in range(2, 251):
    if i not in column_numbers:
        device_columns[f'{i}'] = [None] * len(time_column)

# Re-insert all the columns in sorted order
sorted_columns = {}
for key in sorted(device_columns.keys(), key=lambda x: int(x.split(' ')[0])):
    sorted_columns[key] = device_columns[key]

device_columns = sorted_columns

Combine columns into a csv and export

In [144]:
# Create dataframe with time column and all the device data
df = pd.DataFrame({'1 MILLISEC': time_column, **device_columns}, dtype='Int64')

# Save dataframe as CSV with space separated values
df.to_csv(f'test_data_30_05_{pose_type}.csv', sep=' ', index=False, na_rep='NaN')